Load data for coasters and user preferences

In [ ]:

from cpml.recommendation.data_access import load_table

roller_coaster_df = load_table("ROLLER_COASTERS")
#user_ratings_df = load_table("USER_ROLLER_COASTER_RATINGS")
user_ratings_df = load_table("BOBS_ROLLER_COASTERS")

roller_coaster_df = roller_coaster_df.set_index("id")
original_roller_coaster_ratings = roller_coaster_df.copy()


Show data loaded for verification

In [ ]:
from cpml.recommendation.preprocess import drop_missing_parks

roller_coaster_df = drop_missing_parks(roller_coaster_df)
roller_coaster_df.head()

In [ ]:
user_ratings_df=user_ratings_df[user_ratings_df['user_id']=='bob_molby']

Define features of interest

In [ ]:
from cpml.recommendation.preprocess import add_engineered_features, build_feature_matrix

roller_coaster_df = add_engineered_features(roller_coaster_df)
X = build_feature_matrix(roller_coaster_df)
X.head()


Scale Values

In [ ]:
from cpml.recommendation.preprocess import scale_numeric_features

numeric_feature_columns = [
    "imputed_length",
    "imputed_height",
    "imputed_duration",
    "imputed_speed",
]
X_numeric_scaled, scaler = scale_numeric_features(roller_coaster_df, numeric_feature_columns)

X_numeric_scaled.head()

In [ ]:
X.head()

In [ ]:
from cpml.recommendation.preprocess import combine_scaled_and_categorical

X_final = combine_scaled_and_categorical(X_numeric_scaled, X)

X_final.head()


In [ ]:
len(roller_coaster_df)

Build user profile

In [ ]:
from cpml.recommendation.user_ratings import build_rated_features

rated_df, rated_features = build_rated_features(
    user_ratings_df=user_ratings_df,
    roller_coaster_df=roller_coaster_df,
    X_final=X_final,
)
rated_features.head()

In [ ]:
from cpml.recommendation.user_ratings import build_rated_features
from cpml.recommendation.recommend import build_user_profile, score_all_items, recommend_for_user

# 1) Join ratings with coaster metadata and features
rated_df, rated_features = build_rated_features(
    user_ratings_df=user_ratings_df,
    roller_coaster_df=roller_coaster_df,
    X_final=X_final,
)

# 2) Build user profile vector
user_profile = build_user_profile(rated_df, rated_features)

# 3) Score all coasters vs this profile
similarities = score_all_items(X_final, user_profile)

# 4) Get top recommendations
top_recommendations = recommend_for_user(
    user_ratings_df=user_ratings_df,
    roller_coaster_df=roller_coaster_df,
    country='United States',
    similarities=similarities,
    top_k=100,
)



top_recommendations[['name', 'similarity', 'type', 'design', 'imputed_intensity','amusement_park']].head(100)

In [4]:
from cpml.recommendation.recommendation_engine import RecommendationEngine
import pandas as pd

# 1. Build engine once
engine = RecommendationEngine("roller_coasters")

# 2. Build user ratings df
user_ratings_df = pd.DataFrame([
    {"roller_coaster_id": 1, "rating": 5},
    {"roller_coaster_id": 14, "rating": 1},
])

user_ratings_df = pd.DataFrame([
    {"roller_coaster_id": 4253, "rating": 5},
    {"roller_coaster_id": 67, "rating": 1},
])


user_ratings_df = pd.DataFrame([
    (4253, 4.5),    (14, 4),    (67, 4),    (1, 3.5),    (69, 3.5),    (750, 3),    (3570, 4.5),    (14, 4),
    (13383, 3.5),    (200, 2.5),    (13, 1.5),    (22124, 5),    (10089, 4.5),    (1896, 4),    (15411, 4),
    (2860, 4),    (201, 4),    (2498, 4),    (20342, 3.5),    (10891, 4),    (69, 3.5),    (70, 3.5),    (11437, 4),
    (14116, 4),    (106, 3),    (6640, 3),    (18, 2.5),    (530, 2.5),    (202, 2.5),    (200, 2.5),    (108, 2.5),
    (12, 2),    (73, 3.5),    (66, 2),    (7, 1),
], columns=["roller_coaster_id", "rating"])


# 3. Get personalized recommendations
top_recs = engine.recommend_for_user(['United States','Canada'],user_ratings_df, top_k=20)

# 4. Look at them
top_recs[["name", "amusement_park", "similarity"]].head(10)

# 5. Also: similar to a specific coaster
#similar_to_101 = engine.recommend_similar_to_coaster(101, top_k=20)
#similar_to_101

C:\Users\rober\PycharmProjects\CedarPointMLModel\src\cpml\recommendation\data_access.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {name}", conn)


,name,amusement_park,similarity
id,,,
16812.0,Pantheon,Busch Gardens Williamsburg,0.968013
16985.0,Iron Gwazi,Busch Gardens Tampa Bay,0.942043
20549.0,Stardust Racers,Universal Epic Universe,0.930839
11663.0,Iron Rattler,Six Flags Fiesta Texas,0.943900
6992.0,VelociCoaster,Universal Studios Islands of Adventure,0.926608
11579.0,Cannibal,Lagoon,0.928959
12758.0,Mako,SeaWorld Orlando,0.929869
2662.0,SheiKra,Busch Gardens Tampa Bay,0.958580
3290.0,Goliath,Six Flags Over Georgia,0.930527


In [5]:
top_recs.head()


,name,url,amusement_park,type,design,status,manufacturer,model,length,height,...,imputed_duration,standardized_park_name,exclude_record,standardized_coaster_name,country,average_rating,has_inversions,intensity_level,similarity,score
id,,,,,,,,,,,,,,,,,,,,,
16812.0,Pantheon,/16812.htm,Busch Gardens Williamsburg,Steel,Sit Down,Operating,Intamin Amusement Rides,Custom,3328.0,178.0,...,119.634831,None,None,None,United States,4.70,True,3,0.968013,1.992171
16985.0,Iron Gwazi,/16985.htm,Busch Gardens Tampa Bay,Steel,Sit Down,Operating,Rocky Mountain Construction,Custom,4075.0,206.0,...,110.000000,None,None,None,United States,4.92,True,3,0.942043,1.967740
20549.0,Stardust Racers,/20549.htm,Universal Epic Universe,Steel,Sit Down,Operating,Mack Rides GmbH & Co KG,unknown,NaN,133.0,...,119.634831,None,None,None,United States,4.92,True,3,0.930839,1.944336
11663.0,Iron Rattler,/11663.htm,Six Flags Fiesta Texas,Steel,Sit Down,Operating,Rocky Mountain Construction,Custom,3266.0,179.0,...,119.634831,None,None,None,United States,4.69,True,3,0.943900,1.941224
6992.0,VelociCoaster,/6992.htm,Universal Studios Islands of Adventure,Steel,Sit Down,Operating,Intamin Amusement Rides,Custom,4700.0,155.0,...,119.634831,None,None,None,United States,4.91,True,3,0.926608,1.934201
